In [1]:
import pyodbc
import pandas as pd
import sqlite3
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

In [2]:


DB_SOURCE = {
    'servername': r'LAPTOP-PRQNR6SS\SQLEXPRESS',
    'database': 'SDM'
}


DB_TARGET = {
    'servername': r'LAPTOP-PRQNR6SS\SQLEXPRESS',
    'database': 'Ster_schema'
}

# Maak verbindingen
source_conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    f"SERVER={DB_SOURCE['servername']};"
    f"DATABASE={DB_SOURCE['database']};"
    "Trusted_Connection=yes;"
)

target_conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    f"SERVER={DB_TARGET['servername']};"
    f"DATABASE={DB_TARGET['database']};"
    "Trusted_Connection=yes;"
)

# Cursors maken
source_cursor = source_conn.cursor()
target_cursor = target_conn.cursor()

print("Succesvol verbonden met beide databases!")

Succesvol verbonden met beide databases!


In [3]:
# Haal een lijst van tabellen op uit de bron-database
sql_query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'"


# Gebruik pandas om de query uit te voeren en de tabellen op te halen
tables_df = pd.read_sql(sql_query, source_conn)

# Print de lijst van tabellen
tables_df

,TABLE_NAME
0,returned_item
1,satisfaction_type
2,satisfaction
3,training
4,inventory_levels_train
5,product_forecast_train
6,sales_territory
7,country
8,order_method
9,product_line


## alle query's


In [4]:
product_type_query = "SELECT * FROM product_type"
product_type_df = pd.read_sql(product_type_query, source_conn)

product_line_query = "SELECT * FROM product_line"
product_line_df = pd.read_sql(product_line_query, source_conn)

product_query = "SELECT * FROM product"
product_df = pd.read_sql(product_query, source_conn)

product_forecast_query = "SELECT * FROM product_forecast_train"
product_forecast_df = pd.read_sql(product_forecast_query, source_conn)

order_details_query = "SELECT * FROM order_details"
order_details_df = pd.read_sql(order_details_query, source_conn)

order_header_query = "SELECT * FROM order_header"
order_header_df = pd.read_sql(order_header_query, source_conn)

order_methode_query = "SELECT * FROM order_method"
order_methode_df = pd.read_sql(order_methode_query, source_conn)

returned_item_query = "SELECT * FROM returned_item"
returned_item_df = pd.read_sql(returned_item_query, source_conn)

return_reason_query = "SELECT * FROM return_reason"
return_reason_df = pd.read_sql(return_reason_query, source_conn)

sales_staff_query = "SELECT * FROM sales_staff"
sales_staff_df = pd.read_sql(sales_staff_query, source_conn)

sales_branch_query = "SELECT * FROM sales_branch"
sales_branch_df = pd.read_sql(sales_branch_query, source_conn)

country_query = "SELECT * FROM country"
country_df = pd.read_sql(country_query, source_conn)

sales_territory_query = "SELECT * FROM sales_territory"
sales_territory_df = pd.read_sql(sales_territory_query, source_conn)

retailer_query = "SELECT * FROM retailer"
retailer_df = pd.read_sql(retailer_query, source_conn)

retailer_site_query = "SELECT * FROM retailer_site"
retailer_site_df = pd.read_sql(retailer_site_query, source_conn)

retailer_type_query = "SELECT * FROM retailer_type"
retailer_type_df = pd.read_sql(retailer_type_query, source_conn)

retailer_contact_query = "SELECT * FROM retailer_contact"
retailer_contact_df = pd.read_sql(retailer_contact_query, source_conn)





## DATA INVULLEN: MONTH_dim

In [ ]:
months = [(1, 'January'), (2, 'February'), (3, 'March'), (4, 'April'), (5, 'May'), (6, 'June'), (7, 'July'), (8, 'August'), (9, 'September'), (10, 'October'), (11, 'November'), (12, 'December')]

insert_query = "INSERT INTO Month_dim (MONTH_KEY, MONTH) VALUES (?, ?)"

target_cursor.executemany(insert_query, months)

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

## DATA INVULLEN: DATE_dim

In [50]:

returned_date = returned_item_df['RETURN_DATE']
returned_date = pd.to_datetime(returned_date, format='%b %d %Y %I:%M%p')
returned_date = returned_date.dt.strftime('%Y-%m-%d')
returned_date = pd.to_datetime(returned_date)

product_date = product_df['INTRODUCTION_DATE']
product_date = pd.to_datetime(product_date)

order_date = order_header_df['ORDER_DATE']
order_date = pd.to_datetime(order_date)

sales_staff_date = sales_staff_df['DATE_HIRED']
sales_staff_date = pd.to_datetime(sales_staff_date)


all_dates = pd.concat([returned_date, product_date, order_date, sales_staff_date, sales_staff_date]).drop_duplicates()

# Stap 5: Genereer extra datumvelden
date_dim_df = pd.DataFrame()
date_dim_df['FULL_DATE'] = all_dates
date_dim_df['DATE_KEY'] = date_dim_df['FULL_DATE'].dt.strftime('%Y%m%d').astype(int)
date_dim_df['MONTH_EN'] = date_dim_df['FULL_DATE'].dt.strftime('%B')
date_dim_df['MONTH'] = date_dim_df['FULL_DATE'].dt.month.astype(int)
date_dim_df['QUARTER'] = date_dim_df['FULL_DATE'].dt.quarter.astype(int)
date_dim_df['YEAR'] = date_dim_df['FULL_DATE'].dt.year.astype(int)


for index, row in date_dim_df.iterrows():
    try:
        # Stap 6: Insert in de target database
        insert_query = """
        INSERT INTO Date_dim (DATE_KEY, FULL_DATE, MONTH_EN, MONTH, QUARTER, YEAR)
        VALUES (?, ?, ?, ?, ?, ?)
        """

        target_cursor.execute(
            insert_query,
            row['DATE_KEY'], row['FULL_DATE'],
            row['MONTH_EN'], row['MONTH'],
            row['QUARTER'], row['YEAR'] )


    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {e}")


target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


## DATA INVULLEN: Country

In [66]:
country_merge = pd.merge(country_df, sales_territory_df, how='left', on='SALES_TERRITORY_CODE')
country_merge

,COUNTRY_CODE,COUNTRY,LANGUAGE,CURRENCY_NAME,FLAG_IMAGE,SALES_TERRITORY_CODE,TERRITORY_NAME_EN
0,1,France,EN,francs,F01,6,Central Europe
1,2,Germany,EN,marks,F02,6,Central Europe
2,3,United States,EN,dollars,F03,1,Americas
3,4,Canada,EN,dollars,F04,1,Americas
4,5,Austria,EN,schillings,F05,7,Southern Europe
5,6,Italy,EN,lira,F06,7,Southern Europe
6,7,Netherlands,EN,guilders,F07,5,Northern Europe
7,8,Switzerland,EN,francs,F08,6,Central Europe
8,9,United Kingdom,EN,pounds,F09,6,Central Europe
9,10,Sweden,EN,krona,F10,5,Northern Europe


In [68]:
for index, row in country_merge.iterrows():
    try:
        query = """
        INSERT INTO Country (
            COUNTRY_CODE, COUNTRY, LANGUAGE_EN, CURRENCY_NAME, TERRITORY_NAME_EN
        ) VALUES (?, ?, ?, ?, ?)
        """

        target_cursor.execute(
            query,
            row['COUNTRY_CODE'], row['COUNTRY'], row['LANGUAGE'], row['CURRENCY_NAME'], row['TERRITORY_NAME_EN']
        )
    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {e}")

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")



Data succesvol geïmporteerd naar SSMS!


 ## DATABASE INVULLEN : retailer

In [69]:

retailer_type_merge = pd.merge(retailer_df, retailer_type_df, how='left', on='RETAILER_TYPE_CODE')

retailer_merge = pd.merge(retailer_type_merge, retailer_site_df, how='right', on='RETAILER_CODE')


retailer_merge


,RETAILER_CODE,RETAILER_CODEMR,COMPANY_NAME,RETAILER_TYPE_CODE,RETAILER_TYPE_EN,RETAILER_SITE_CODE,ADDRESS1,ADDRESS2,CITY,REGION,COUNTRY_CODE,ACTIVE_INDICATOR
0,89,1.0,Rock Steady,6,Outdoors Shop,1,1117 Franklin Blvd,None,Winnipeg,Manitoba,4,1
1,89,1.0,Rock Steady,6,Outdoors Shop,2,"45, rue Atwater",None,Montréal,Québec,4,1
2,89,1.0,Rock Steady,6,Outdoors Shop,3,328 Hodgson Road,None,Fredericton,New Brunswick,4,1
3,90,2.0,Ultra Sports,8,Sports Store,4,"2425, rue Peel",None,Montréal,Québec,4,1
4,90,2.0,Ultra Sports,8,Sports Store,5,34 8th Avenue,None,Sudbury,Ontario,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
386,209,415.0,Ausrüstungshaus Globetrotter,2,Department Store,439,Hüttenstraße 87,None,Klosterneuburg,None,5,1
387,209,415.0,Ausrüstungshaus Globetrotter,2,Department Store,440,Hauptstraße 85,None,Linz,None,5,1
388,209,415.0,Ausrüstungshaus Globetrotter,2,Department Store,441,Amselweg 8,None,Innsbruck,None,5,1
389,209,415.0,Ausrüstungshaus Globetrotter,2,Department Store,442,Hasengasse 38,None,Ischgl,None,5,1


In [ ]:

for index, row in retailer_merge.iterrows():
    try:
        query = """
        INSERT INTO retailer (
            RETAILER_SITE_CODE, RETAILER_CODE, COMPANY_NAME, ADDRESS1, ADDRESS2, CITY, REGION, ACTIVE_INDICATOR, RETAILER_TYPE_EN, COUNTRY_CODE
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """

        target_cursor.execute(
            query,
            row['RETAILER_SITE_CODE'], row['RETAILER_CODE'], row['COMPANY_NAME'], row['ADDRESS1'], row['ADDRESS2'], row['CITY'], row['REGION'], row['ACTIVE_INDICATOR'], row['RETAILER_TYPE_EN'], row['COUNTRY_CODE']
        )
    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {e}")

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")




## DATABASE INVULLEN: Retailer_contact

In [78]:

for index, row in retailer_contact_df.iterrows():
    try:
        query = """
        INSERT INTO Retailer_contact (
            RETAILER_CONTACT_CODE, RETAILER_SITE_CODE, FULL_NAME, JOB_POSITION_EN, GENDER
        ) VALUES (?, ?, ?, ?, ?)
        """
        target_cursor.execute(
            query,
            row['RETAILER_CONTACT_CODE'], row['RETAILER_SITE_CODE'],row['FIRST_NAME'] + ' ' + row['LAST_NAME'], row['JOB_POSITION_EN'], row['GENDER']
        )

    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {e}")

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")




Data succesvol geïmporteerd naar SSMS!


## DATABASE INVULLEN: Sales_staff

In [79]:
sales_merge = pd.merge(sales_staff_df, sales_branch_df, how='left', on='SALES_BRANCH_CODE')

sales_merge

,SALES_STAFF_CODE,FIRST_NAME,LAST_NAME,POSITION_EN,WORK_PHONE,EXTENSION,FAX,EMAIL,DATE_HIRED,SALES_BRANCH_CODE,MANAGER_CODE,ADDRESS1,ADDRESS2,CITY,REGION,COUNTRY_CODE
0,4,Denis,Pagé,Branch Manager,+33 1 68 94 52 20,325.0,+33 1 68 94 56 60,DPage@grtd123.com,1996-11-03,6,18,"75, rue du Faubourg St-Honoré",None,Paris,None,1
1,5,Élizabeth,Michel,Level 3 Sales Representative,+33 1 68 94 52 20,336.0,+33 1 68 94 56 60,EMichel@grtd123.com,1995-06-08,6,4,"75, rue du Faubourg St-Honoré",None,Paris,None,1
2,6,Émile,Clermont,Level 1 Sales Representative,+33 1 68 94 52 20,378.0,+33 1 68 94 56 60,EClermont@grtd123.com,1998-04-07,6,4,"75, rue du Faubourg St-Honoré",None,Paris,None,1
3,7,Étienne,Jauvin,Level 2 Sales Representative,+33 1 68 94 52 20,398.0,+33 1 68 94 56 60,EJauvin@grtd123.com,1997-08-16,6,4,"75, rue du Faubourg St-Honoré",None,Paris,None,1
4,12,Elsbeth,Wiesinger,Level 2 Sales Representative,+(49) 40 663 1990,1818.0,+(49) 40 663 4571,EWiesinger@grtd123.com,1997-02-27,13,14,Schwabentor 35,None,Hamburg,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,120,Giele,Laermans,Level 1 Sales Representative,+32 16 20.73.21,1340.0,+32 16 20.73.32,GLaermans@grtd123.com,1999-11-15,38,119,Interleuvenlaan 2,None,Heverlee,None,17
98,121,François,De Crée,Level 1 Sales Representative,+32 16 20.73.21,1642.0,+32 16 20.73.32,FDecree@grtd123.com,1999-12-01,38,119,Interleuvenlaan 2,None,Heverlee,None,17
99,122,Yvette,Lattrez,Level 3 Sales Representative,+32 16 20.73.21,1633.0,+32 16 20.73.32,YLattrez@grtd123.com,1999-12-09,38,119,Interleuvenlaan 2,None,Heverlee,None,17
100,123,Willi,Seefelder,Level 2 Sales Representative,+(43) 13 79 56 32,325.0,+(43) 13 79 56 33,WSeefelder@grtd123.com,1998-10-28,39,18,Jedleser Straße 7,None,Wien,None,5


In [ ]:

for index, row in sales_merge.iterrows():
    try:
        query = """
        INSERT INTO Sales_staff (
            SALES_STAFF_CODE, FULL_NAME, POSITION_EN, DATE_HIRED_ID, MANAGER_CODE, ADDRESS1, ADDRESS2, CITY, REGION, COUNTRY_CODE
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        date_hired = pd.to_datetime(row['DATE_HIRED'])
        date_hired = int(date_hired.strftime('%Y%m%d'))

        target_cursor.execute(
            query,
            row['SALES_STAFF_CODE'], row['FIRST_NAME'] + ' ' + row['LAST_NAME'] , row['POSITION_EN'], date_hired, row['MANAGER_CODE'], row['ADDRESS1'], row['ADDRESS2'], row['CITY'], row['REGION'], row['COUNTRY_CODE']
        )

    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {e}")

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


## DATABASE INVULLEN: Product

In [83]:
type_line_merge = pd.merge(product_type_df, product_line_df, how='left', on='PRODUCT_LINE_CODE')

product_merge = pd.merge(product_df, type_line_merge, how='left', on='PRODUCT_TYPE_CODE')
product_merge

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,PRODUCT_LINE_CODE,PRODUCT_TYPE_EN,PRODUCT_LINE_EN
0,1,1995-02-15,1,4.000000,0.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",1,Cooking Gear,Camping Equipment
1,2,1995-02-15,1,9.220000,0.23,P02CE1CG1.jpg,EN,TrailChef Canteen,Aluminum canteen. Rugged fleece-lined cover wi...,1,Cooking Gear,Camping Equipment
2,3,1995-02-15,1,15.930000,0.28,P03CE1CG1.jpg,EN,TrailChef Kitchen Kit,"Zippered nylon pouch contains cutlery for two,...",1,Cooking Gear,Camping Equipment
3,4,1995-02-15,1,5.000000,0.28,P04CE1CG1.jpg,EN,TrailChef Cup,Tin cup. Holds 0.4 liters. Weight: 60 g,1,Cooking Gear,Camping Equipment
4,5,1995-02-15,1,34.970001,0.30,P05CE1CG1.jpg,EN,TrailChef Cook Set,All you will ever need on the trail. Pot grip...,1,Cooking Gear,Camping Equipment
...,...,...,...,...,...,...,...,...,...,...,...,...
110,111,2003-12-15,20,81.800003,0.55,P111GE5PT20.jpg,EN,Blue Steel Max Putter,Putter head is composed from a single piece of...,5,Putters,Golf Equipment
111,112,2004-01-10,21,6.000000,0.33,P112GE5GA21.jpg,EN,Course Pro Golf and Tee Set,Set includes two premium golf ball and an asso...,5,Golf Accessories,Golf Equipment
112,113,2004-01-15,21,6.000000,0.50,P113GE5GA21.jpg,EN,Course Pro Umbrella,Large red and white patterned golf umbrella.,5,Golf Accessories,Golf Equipment
113,114,2003-12-15,21,80.000000,0.60,P114GE5GA21.jpg,EN,Course Pro Golf Bag,High quality golf bag that includes carrying s...,5,Golf Accessories,Golf Equipment


In [ ]:


for index, row in product_merge.iterrows():

    try:
    #
        query = """
        INSERT INTO product (PRODUCT_NUMBER, INTRODUCTION_DATE_ID, PRODUCTION_COST, MARGIN, PRODUCT_NAME, PRODUCT_TYPE_EN, PRODUCT_DESCRIPTION, PRODUCT_LINE_EN)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """

        introduction_date = pd.to_datetime(row['INTRODUCTION_DATE'])
        introduction_date = int(introduction_date.strftime('%Y%m%d'))

        target_cursor.execute(query,
                              row["PRODUCT_NUMBER"],  introduction_date,
                              row["PRODUCTION_COST"],
                              row["MARGIN"],
                              row["PRODUCT_NAME"],
                              row["PRODUCT_TYPE_EN"], row["DESCRIPTION"], row["PRODUCT_LINE_EN"])
    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {row}")
        print(f"Foutmelding: {e}")

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

## DATABASE INVULLEN: Fact_Orders

In [5]:
order_header_methode_merge = pd.merge(order_header_df, order_methode_df, how='left', on='ORDER_METHOD_CODE')

orders_merge = pd.merge(order_header_methode_merge, order_details_df, how='left', on='ORDER_NUMBER')

orders_merge

,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,SALES_STAFF_CODE,SALES_BRANCH_CODE,ORDER_DATE,ORDER_METHOD_CODE,ORDER_METHOD_EN,ORDER_DETAIL_CODE,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,1153,Ultra Sports,4,6,50,18,2022-04-14,7,Sales visit,47747,2,100,9.640000,12.530000,11.810000
1,1153,Ultra Sports,4,6,50,18,2022-04-14,7,Sales visit,47994,32,254,18.250000,27.370001,25.100000
2,1153,Ultra Sports,4,6,50,18,2022-04-14,7,Sales visit,48015,35,114,19.670000,26.540001,24.820000
3,1153,Ultra Sports,4,6,50,18,2022-04-14,7,Sales visit,48022,36,130,21.030001,34.900002,31.440001
4,1153,Ultra Sports,4,6,50,18,2022-04-14,7,Sales visit,48049,39,74,42.330002,67.730003,61.380001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37752,9479,Preben's T°y,338,329,27,15,2024-02-02,5,Web,110788,52,64,19.000000,38.000000,33.250000
37753,9479,Preben's T°y,338,329,27,15,2024-02-02,5,Web,110816,56,24,9.000000,18.000000,15.750000
37754,9479,Preben's T°y,338,329,27,15,2024-02-02,5,Web,110827,57,40,40.000000,80.000000,70.000000
37755,9479,Preben's T°y,338,329,27,15,2024-02-02,5,Web,110914,78,32,79.190002,99.000000,94.050003


In [ ]:
for index, row in orders_merge.iterrows():
    try:
        query = """
        INSERT INTO fact_orders (
            ORDER_NUMBER, ORDER_DATE_ID, RETAILER_SITE_CODE, PRODUCT_NUMBER, SALES_STAFF_CODE, ORDER_METHOD_EN, QUANTITY, UNIT_COST, UNIT_PRICE, UNIT_SALE_PRICE, TOTAL_AMOUNT_TURNOVER, PROFIT
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """
        orders_date = pd.to_datetime(row['ORDER_DATE'])
        orders_date = int(orders_date.strftime('%Y%m%d'))

        # Berekening van omzet
        omzet = row['QUANTITY'] * row['UNIT_SALE_PRICE']

        # Berekening van winst
        winst = (row['UNIT_SALE_PRICE'] - row['UNIT_COST']) * row['QUANTITY']

        target_cursor.execute(
            query,
            row['ORDER_NUMBER'], orders_date, row['RETAILER_SITE_CODE'], row['PRODUCT_NUMBER'], row['SALES_STAFF_CODE'], row['ORDER_METHOD_EN'], row['QUANTITY'], row['UNIT_COST'], row['UNIT_PRICE'], row['UNIT_SALE_PRICE'], omzet, winst
        )

    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {e}")

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")

## DATABASE INVULLEN: Fact_returned_item

In [7]:
returned_item_merge = pd.merge(returned_item_df, return_reason_df, how='left', on='RETURN_REASON_CODE')

return_merge = pd.merge(returned_item_merge, orders_merge, how='left', on='ORDER_DETAIL_CODE')

return_merge

,RETURN_CODE,RETURN_DATE,ORDER_DETAIL_CODE,RETURN_REASON_CODE,RETURN_QUANTITY,RETURN_DESCRIPTION_EN,ORDER_NUMBER,RETAILER_NAME,RETAILER_SITE_CODE,RETAILER_CONTACT_CODE,SALES_STAFF_CODE,SALES_BRANCH_CODE,ORDER_DATE,ORDER_METHOD_CODE,ORDER_METHOD_EN,PRODUCT_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,1491,Aug 1 2023 4:10AM,84858,5,8,Unsatisfactory product,7446,Ausr³stungshaus Globetrotter,440,459,123,39,2023-07-23,2,Telephone,6,8,87.629997,123.559998,123.559998
1,1492,Dec 6 2022 6:46PM,84440,2,2,Incomplete product,7316,Ausr³stungshaus Globetrotter,441,460,124,39,2022-10-18,7,Sales visit,8,22,88.230003,151.770004,151.770004
2,1493,Jun 24 2023 9:23AM,84867,4,22,Wrong product shipped,7359,Ausr³stungshaus Globetrotter,442,461,124,39,2023-06-12,3,Mail,8,22,78.260002,134.630005,134.630005
3,1494,Jul 21 2023 12:00AM,84873,3,20,Wrong product ordered,7448,Outdoor-Fachgeschõft M³ller,435,454,124,39,2023-07-16,7,Sales visit,8,20,78.260002,134.630005,126.160004
4,1496,Oct 7 2022 5:13AM,84488,1,2,Defective product,7438,Outdoor-Fachgeschõft M³ller,438,457,123,39,2022-08-23,2,Telephone,17,28,64.360001,90.089996,90.089996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,2464,Dec 7 2023 5:14AM,114967,1,12,Defective product,7623,Esportes Grumari,335,320,98,30,2023-10-14,7,Sales visit,63,128,30.629999,45.970001,38.310001
615,2465,Jun 6 2023 9:22AM,114972,3,24,Wrong product ordered,7625,Esportes Grumari,335,320,98,30,2023-05-17,2,Telephone,66,24,46.450001,92.910004,69.680000
616,2466,Oct 6 2023 12:31PM,114983,5,66,Unsatisfactory product,7622,Esportes Grumari,335,320,98,30,2023-09-26,5,Web,71,66,60.500000,90.760002,75.629997
617,2467,Mar 31 2024 3:39PM,115215,4,64,Wrong product shipped,7630,Esportes Grumari,335,320,96,30,2024-03-13,2,Telephone,73,64,8.560000,11.980000,10.270000


In [11]:
for index, row in return_merge.iterrows():
    try:
        query = """
        INSERT INTO fact_returned_item (
            RETURN_CODE, RETURN_DATE_ID, PRODUCT_NUMBER, RETAILER_SITE_CODE, SALES_STAFF_CODE, RETURN_QUANTITY, RETURN_DESCRIPTION_EN
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
        """

        # Converteer retourdatum naar een integer in YYYYMMDD-formaat
        return_date = pd.to_datetime(row['RETURN_DATE'], format='%b %d %Y %I:%M%p')
        return_date = int(return_date.strftime('%Y%m%d'))

        # SQL-query uitvoeren met de juiste waarden
        target_cursor.execute(
            query,
            row['RETURN_CODE'], return_date, row['PRODUCT_NUMBER'], row['RETAILER_SITE_CODE'], row['SALES_STAFF_CODE'], row['RETURN_QUANTITY'], row['RETURN_DESCRIPTION_EN']
        )

    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {e}")

target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!


 ## DATABASE INVULLEN : product_forecast

In [12]:
import uuid  # Voor unieke ID's

# Loop door de rijen en voer de INSERT uit
for index, row in product_forecast_df.iterrows():
    try:
        # Cast de waarden expliciet naar de juiste types
        product_number = int(row["PRODUCT_NUMBER"])
        expected_volume = int(row["EXPECTED_VOLUME"])
        year = int(row["YEAR"])
        month_id = int(row["MONTH"])

        # Genereer een unieke ID (UUID of combinatie van waardes)
        product_forecast_id = f"{product_number}{year}{month_id}"

        # Query voor de nieuwe tabel
        query = """
        INSERT INTO fact_product_forecast (PRODUCT_FORECAST_ID, PRODUCT_NUMBER, EXPECTED_VOLUME, YEAR, MONTH_ID)
        VALUES (?, ?, ?, ?, ?)
        """

        # Voer de query uit
        target_cursor.execute(query, (product_forecast_id, product_number, expected_volume, year, month_id))

    except pyodbc.Error as e:
        print(f"Fout bij rij {index}: {row}")
        print(f"Foutmelding: {e}")

# Commit de wijzigingen
target_conn.commit()
print("Data succesvol geïmporteerd naar SSMS!")


Data succesvol geïmporteerd naar SSMS!
